This concept notebook is an attempt to create a "worst case scenario" for aperture photometry in Jdaviz.

See https://github.com/spacetelescope/jdaviz/issues/2139#issuecomment-1507619222

In [ ]:
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.utils.data import get_pkg_data_filename
from regions import PixCoord, CirclePixelRegion, EllipsePixelRegion, RectanglePixelRegion

from jdaviz import Imviz

%matplotlib inline

In [ ]:
imviz = Imviz()
imviz.show()

### Image 1: A scene with regular WCS

A hundred Gaussian objects with FITS WCS. This is the data used in `photutils/aperture/tests/test_stats.py`.

In [ ]:
file1 = get_pkg_data_filename('data/gauss100_fits_wcs.fits', package='jdaviz.configs.imviz.tests')
imviz.load_data(file1)

We will use circular apertures from the test case for now so we can check against answers from `photutils`.

In [ ]:
regions = []
positions = [(145.1, 168.3), (48.3, 200.3)]
for x, y in positions:
    regions.append(CirclePixelRegion(center=PixCoord(x=x, y=y), radius=5))

Add elliptical and rectangular regions too so we can study any asymetrical effects on Subset.

In [ ]:
regions += [
    EllipsePixelRegion(center=PixCoord(x=84.7, y=224.1), width=23, height=9, angle=2.356 * u.rad),
    RectanglePixelRegion(center=PixCoord(x=229, y=152), width=17, height=7)
]

In [ ]:
imviz.load_regions(regions)

### Image 2: WCS with a different pixel scale

Now we need an image would result in the Subset being hard to work with. Say, this image would have a different pixel scale. Offset of the center is also implicitly tested by using a different pixel scale.

*Note: Originally, distortion was also planned but it added a huge amount of computation time to reproject and linking; hence it was abandoned from the test case.*

We define a different pixel scale by down-sampling it by half each dimension.

In [ ]:
file2 = get_pkg_data_filename('data/gauss100_fits_wcs_block_reduced.fits', package='jdaviz.configs.imviz.tests')
imviz.load_data(file2)

### Image 3: Add rotation to downsampled image

Like above but with extra rotation.

In [ ]:
file3 = get_pkg_data_filename('data/gauss100_fits_wcs_block_reduced_rotated.fits', package='jdaviz.configs.imviz.tests')
imviz.load_data(file3)

### Link them by WCS

In [ ]:
imviz.link_data(link_type='wcs')

### How is glue projecting the rectangle?

How is `glue` really projecting the rectangle for all the data? This is without any special handling using sky coordinates. Also, this is not the mask that is getting passed into `photutils`.

In [ ]:
rect_grp = imviz.app.data_collection.subset_groups[3]

In [ ]:
fig, axs = plt.subplots(1, 3)
axs[0].imshow(rect_grp.subsets[0].to_mask(), origin='lower')
axs[0].set_xlim(210, 250)
axs[0].set_ylim(140, 160)
axs[1].imshow(rect_grp.subsets[1].to_mask(), origin='lower')
axs[1].set_xlim(105, 125)
axs[1].set_ylim(70, 80)
axs[2].imshow(rect_grp.subsets[2].to_mask(), origin='lower')
axs[2].set_xlim(105, 125)
axs[2].set_ylim(70, 80);

Even when unrotated, a different pixel scale does affect the projected mask dimension. When rotated, it is not even a rectangle anymore; furthermore, even though this projection follows WCS linking, the array from `to_mask()` does not account for rotation that we see in the viewer.

### What is aperture photometry telling us now?

In [ ]:
phot_plugin = imviz.plugins["Aperture Photometry"]._obj

To re-calculate for a different subset, uncomment the desired Subset and re-run affected cells.

In [ ]:
phot_plugin.aperture_selected = "Subset 1"
#phot_plugin.aperture_selected = "Subset 2"
#phot_plugin.aperture_selected = "Subset 3"
#phot_plugin.aperture_selected = "Subset 4"

Calculate for the same Subset for all the data.

Original data has a mean background of 5. Block-reduced data will have mean background of 20 (4 pixels combined into 1 pixel, while preserving total flux).

In [ ]:
phot_plugin.background_selected = 'Manual'
phot_plugin.background_value = 5.0

phot_plugin.dataset_selected = "gauss100_fits_wcs[PRIMARY,1]"
phot_plugin.vue_do_aper_phot()

phot_plugin.background_value = 20.0

phot_plugin.dataset_selected = "gauss100_fits_wcs_block_reduced[PRIMARY,1]"
phot_plugin.vue_do_aper_phot()

phot_plugin.dataset_selected = "gauss100_fits_wcs_block_reduced_rotated[PRIMARY,1]"
phot_plugin.vue_do_aper_phot()

Look at the results. The aperture sum should be close (within 10%) to the following numbers regardless of data if Subset is handled correctly:

| Subset name | Aperture sum |
| --- | --- |
| Subset 1 | 738.8803424408962 |
| Subset 2 | 857.5194857987592 |
| Subset 3 | 472.17364321556005 |
| Subset 4 | 837.0023608207703 |

In [ ]:
tbl = imviz.get_aperture_photometry_results()

In [ ]:
tbl["data_label", "subset_label", "sum"]